In [1]:
import boto3, json, time, zipfile, io, os

ENDPOINT = "http://localhost:4566"
REGION = "us-east-1"

# Clients
sqs = boto3.client("sqs", region_name=REGION, endpoint_url=ENDPOINT)
s3 = boto3.client("s3", region_name=REGION, endpoint_url=ENDPOINT)
lambda_client = boto3.client("lambda", region_name=REGION, endpoint_url=ENDPOINT)
iam = boto3.client("iam", region_name=REGION, endpoint_url=ENDPOINT)

BUCKET_NAME = "results-bucket"
TASK_QUEUE = "task-queue"
NOTIFY_QUEUE = "notification-queue"

In [2]:
# Create S3
s3.create_bucket(Bucket=BUCKET_NAME)

# Create Queues
t_q = sqs.create_queue(QueueName=TASK_QUEUE)
n_q = sqs.create_queue(QueueName=NOTIFY_QUEUE)
TASK_QUEUE_URL = t_q["QueueUrl"]
NOTIFY_QUEUE_URL = n_q["QueueUrl"]

# Get ARNs for the Trigger
TASK_QUEUE_ARN = sqs.get_queue_attributes(QueueUrl=TASK_QUEUE_URL, AttributeNames=["QueueArn"])["Attributes"]["QueueArn"]

# Create IAM Role (Wait a second after this)
try:
    role = iam.create_role(
        RoleName="lambda-role",
        AssumeRolePolicyDocument=json.dumps({
            "Version": "2012-10-17",
            "Statement": [{"Effect": "Allow", "Principal": {"Service": "lambda.amazonaws.com"}, "Action": "sts:AssumeRole"}]
        })
    )
    ROLE_ARN = role["Role"]["Arn"]
except:
    ROLE_ARN = "arn:aws:iam::000000000000:role/lambda-role"

print(f"✅ Infrastructure Ready. Task Queue: {TASK_QUEUE_URL}")

✅ Infrastructure Ready. Task Queue: http://sqs.us-east-1.localhost.localstack.cloud:4566/000000000000/task-queue


In [3]:
def cleanup_lambdas():
    functions = ["task_lambda", "notification_lambda"]
    for func in functions:
        try:
            lambda_client.delete_function(FunctionName=func)
            print(f"🗑️ Deleted existing function: {func}")
        except:
            print(f"ℹ️ Function {func} did not exist, skipping delete.")

cleanup_lambdas()
# Now call your deploy function
build_and_deploy()

🗑️ Deleted existing function: task_lambda
🗑️ Deleted existing function: notification_lambda


NameError: name 'build_and_deploy' is not defined

In [4]:
def build_zip_bytes():
    zip_buffer = io.BytesIO()
    with zipfile.ZipFile(zip_buffer, "w", zipfile.ZIP_DEFLATED) as zf:
        # 1. Add the 'app' directory (Your 3-4 real functions)
        for root, _, files in os.walk("../app"):
            for file in files:
                full_path = os.path.join(root, file)
                # This ensures the folder structure is preserved inside the zip
                archive_name = os.path.relpath(full_path, "..")
                zf.write(full_path, archive_name)
        
        # 2. Add the 'lambdas' directory
        for root, _, files in os.walk("../lambdas"):
            for file in files:
                full_path = os.path.join(root, file)
                archive_name = os.path.relpath(full_path, "..")
                zf.write(full_path, archive_name)

    return zip_buffer.getvalue()

In [5]:
def force_deploy(name, zip_bytes, handler, role):
    # 1. Force Delete if exists
    try:
        lambda_client.delete_function(FunctionName=name)
        print(f"🗑️ Deleted existing function: {name}")
        # Small sleep to let LocalStack's internal state catch up
        time.sleep(1)
    except:
        pass # Function didn't exist, which is fine

    # 2. Create Fresh Function
    try:
        lambda_client.create_function(
            FunctionName=name,
            Runtime="python3.9",
            Role=role,
            Handler=handler,
            Code={"ZipFile": zip_bytes},
            Environment={'Variables': {'AWS_ENDPOINT_URL': 'http://172.17.0.1:4566'}}
        )
        print(f"✨ Created FRESH function: {name}")
        
        # 3. Re-link the SQS trigger for the task lambda
        if name == "task_lambda":
            lambda_client.create_event_source_mapping(
                EventSourceArn=TASK_QUEUE_ARN,
                FunctionName=name,
                BatchSize=1
            )
            print(f"🔗 Linked SQS trigger to {name}")
            
    except Exception as e:
        print(f"❌ Failed to deploy {name}: {e}")

# Now run it
zip_content = build_zip_bytes() 
force_deploy("task_lambda", zip_content, "lambdas.task_lambda.lambda_handler", ROLE_ARN)
force_deploy("notification_lambda", zip_content, "lambdas.notification_lambda.lambda_handler", ROLE_ARN)

✨ Created FRESH function: task_lambda
❌ Failed to deploy task_lambda: An error occurred (ResourceConflictException) when calling the CreateEventSourceMapping operation: An event source mapping with SQS arn (" arn:aws:sqs:us-east-1:000000000000:task-queue ") and function (" task_lambda ") already exists. Please update or delete the existing mapping with UUID 35fc4e51-ddc5-4983-b464-313d04e11448
✨ Created FRESH function: notification_lambda


In [6]:
import uuid

# Generate a unique ID for this test
test_id = str(uuid.uuid4())[:8]

test_payload = {
    "task_id": f"task-{test_id}",
    "numbers": [10, 20, 30, 40] # Total should be 100
}

# 1. Send message to the first queue
print(f"📤 Sending Task {test_payload['task_id']} to SQS...")
sqs.send_message(
    QueueUrl=TASK_QUEUE_URL,
    MessageBody=json.dumps(test_payload)
)

# 2. Wait for Lambda to process
print("⏳ Waiting 5 seconds for background processing...")
time.sleep(5)

# 3. Check S3 for the result file
print("🔍 Checking S3...")
try:
    s3_files = s3.list_objects_v2(Bucket=BUCKET_NAME)
    if 'Contents' in s3_files:
        print(f"✅ Success! Found {len(s3_files['Contents'])} files in S3.")
        for obj in s3_files['Contents']:
            print(f"📄 File: {obj['Key']}")
            # Read the content to verify the math
            content = s3.get_object(Bucket=BUCKET_NAME, Key=obj['Key'])
            data = json.loads(content['Body'].read().decode())
            print(f"   📊 Data Content: {data}")
    else:
        print("❓ No files in S3 yet. The Lambda might be crashing.")
except Exception as e:
    print(f"❌ Error checking S3: {e}")

# 4. Check the Notification Queue
print("\n🔍 Checking Notification Queue...")
receive = sqs.receive_message(QueueUrl=NOTIFY_QUEUE_URL, MaxNumberOfMessages=1)
if 'Messages' in receive:
    print(f"✅ Success! Notification received: {receive['Messages'][0]['Body']}")
else:
    print("❓ No notification found in the second queue.")

📤 Sending Task task-6835b6da to SQS...
⏳ Waiting 5 seconds for background processing...
🔍 Checking S3...
❓ No files in S3 yet. The Lambda might be crashing.

🔍 Checking Notification Queue...
❓ No notification found in the second queue.
